In [ ]:
#https://plotly.com/python/visualizing-mri-volume-slices/

In [ ]:
#https://stackoverflow.com/questions/73726025/how-could-i-create-a-fps-1-mp4-video-using-ffmpegwriter-from-matplotlib-animatio

In [ ]:
#idea: extract LUT from Nikon elements viewer, take nd2 as numpy array. Plot frame by frame

In [ ]:
# maybe use instead of mpeg, gif: https://stackoverflow.com/questions/753190/programmatically-generate-video-or-animated-gif-in-python

In [ ]:
# IMPORTANT: https://stackoverflow.com/questions/30965355/speedup-matplotlib-animation-to-video-file

In [ ]:
#Auto-reload modules (used to develop functions outside this notebook)
%load_ext autoreload
%autoreload 2

In [ ]:
import h5py  # TODO: sometimes, importing from two_photon_session causes kernel to crash. In Anaconda prompt, mismatch of 
# h5py and hdf5:  h5py is running against HDF5 1.10.7 when it was built against 1.10.6,
from labrotation.two_photon_session import nb_view_patches_with_lfp_movement, TwoPhotonSession

In [ ]:
import labrotation.file_handling as fh
#from labrotation.two_photon_session import TwoPhotonSession as TPS
from nd2_to_caiman import np_arr_from_nd2, np_arr_and_time_stamps_from_nd2
import numpy as np
import matplotlib.pylab as plt
import matplotlib.animation as animation
import subprocess
import csv
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.font_manager as fm
from matplotlib import pyplot
import os
import pims_nd2

# Play two videos in parallel, below them time series with sliding vertical bar matching current frame of videos
### IMPORTANT: ffmpeg has to be installed, both the python library and the Windows software! https://ffmpeg.org/download.html Example version (that worked with this script): 

FFmpeg 64-bit static Windows build from www.gyan.dev

Version: 4.3.2-2021-02-20-full_build-www.gyan.dev

In [ ]:
ts_left_fname = fh.open_file("Open data for left time series plot")
ts_right_fname = fh.open_file("Open data for right time series plot")

In [ ]:
video_left = fh.open_file("Select nd2 file to put on left")
video_right = fh.open_file("Select nd2 file to put on right")

In [ ]:
#frames_range_left = [8400, 9400]  # first and last frames (1-indexing) to be read
#frames_range_right = [8400, 9400]
frames_range_left = [1, 500]
frames_range_right = [1, 500]
print(frames_range_left[1] - frames_range_left[0] + 1)
print(frames_range_right[1] - frames_range_right[0] + 1)

In [ ]:
ndarr_left, tstamps_left = np_arr_and_time_stamps_from_nd2(video_left, frames_range_left)
ndarr_right, tstamps_right = np_arr_and_time_stamps_from_nd2(video_right, frames_range_right)

In [ ]:
time_series_left = []
with open(ts_left_fname) as csvfile:
    r = csv.reader(csvfile, delimiter=' ')
    i_frame = 0
    for row in r:
        if i_frame in range(frames_range_left[0], frames_range_left[1]+1):
            time_series_left.append(float(row[0].replace(',', '.')))
        i_frame += 1
    
    
time_series_right = []
with open(ts_right_fname) as csvfile:
    r = csv.reader(csvfile, delimiter=' ')
    i_frame = 0
    for row in r:
        if i_frame in range(frames_range_right[0], frames_range_right[1]+1):
            time_series_right.append(float(row[0].replace(',', '.')))
        i_frame += 1
    

In [ ]:
# time_series_left = np.random.rand(len(ndarr_left))
# time_series_right = np.random.rand(len(ndarr_right))

In [ ]:
tstamps_left_zeroed_s = [(tstamps_left[i] - tstamps_left[0])/1000. for i in range(len(tstamps_left))]
tstamps_right_zeroed_s = [(tstamps_right[i] - tstamps_right[0])/1000. for i in range(len(tstamps_right))]

In [ ]:
min_ts_left = min(time_series_left)
min_ts_right = min(time_series_right)
time_series_left_zeroed = [time_series_left[i] - min_ts_left for i in range(len(time_series_left))]
time_series_right_zeroed = [time_series_right[i] - min_ts_right for i in range(len(time_series_right))]

## Adjust parameters for layout, test on single frame here

In [ ]:
i_frame = 10
fontprops = fm.FontProperties(size=18)
#set up the figure

# height ratios:
# https://stackoverflow.com/questions/10388462/matplotlib-different-size-subplots
# update matplotlib, then height_ratios can be directly passed to subplots()!
fig, axs = plt.subplots(2, 2, figsize=(18,10), gridspec_kw={'height_ratios': [3, 1]}, sharey="row")
axs[0,0].axis('off')
axs[0,1].axis('off')
canvas_width, canvas_height = fig.canvas.get_width_height()

# 1.579 µm / pixel -> 100 µm = 63.33 pixels

scalebar1 = AnchoredSizeBar(axs[0,0].transData,
                           63.3, '100 µm', 'lower right', 
                           pad=0.1,
                           color='white',
                           frameon=False,
                           size_vertical=1,
                           fontproperties=fontprops)

scalebar2 = AnchoredSizeBar(axs[0,1].transData,
                           63.3, '100 µm', 'lower right', 
                           pad=0.1,
                           color='white',
                           frameon=False,
                           size_vertical=1,
                           fontproperties=fontprops)


axs[0,0].add_artist(scalebar1)
axs[0,1].add_artist(scalebar2)

# First frame
fig_left = axs[0, 0].imshow(ndarr_left[i_frame], ) #cmap="Greens_r"
ts_left = axs[1,0].plot(tstamps_left_zeroed_s, time_series_left_zeroed)
vline_left = axs[1,0].axvline(x=tstamps_left_zeroed_s[i_frame], color="red")


fig_right = axs[0, 1].imshow(ndarr_right[i_frame], )  #cmap="Greens_r"
ts_right = axs[1,1].plot(tstamps_right_zeroed_s, time_series_right_zeroed)
vline_right = axs[1, 1].axvline(x=tstamps_right_zeroed_s[i_frame], color="red")


axs[0,1].set_title('AAV1.syn.Flex.GCaMP6s.WPRE.SV40\nAAV.CamKIIO.4.Cre SV40', y=1.0, pad=-42, color="white", fontsize=20)
axs[0,0].set_title('AAV1.syn.GCaMP6s.WPRE.SV40', y=1.0, pad=-20, color="white", fontsize=20)

axs[1,0].set_xlabel("Time (s)", fontsize=22)
axs[1,0].tick_params(axis='x', labelsize=20)

axs[1,1].set_xlabel("Time (s)", fontsize=22)
axs[1,1].tick_params(axis='x', labelsize=20)

axs[1,0].set_ylabel("100% df/f", fontsize=18)
axs[1,1].set_ylabel("100% df/f", fontsize=18)
#axs[1,0].tick_params(axis='y', labelsize=20)

#axs[1,0].get_yaxis().set_visible(False)
#axs[1,1].get_yaxis().set_visible(False)
axs[1,0].get_yaxis().set_ticks([])
axs[1,1].get_yaxis().set_ticks([])
axs[1,0].spines['left'].set_visible(False)
axs[1,0].spines['right'].set_visible(False)
axs[1,0].spines['top'].set_visible(False)
axs[1,1].spines['left'].set_visible(False)
axs[1,1].spines['right'].set_visible(False)
axs[1,1].spines['top'].set_visible(False)
axs[1,0].add_patch(pyplot.Rectangle((-3,0),0.5, 1,facecolor='black',
                              clip_on=False,linewidth = 0))
axs[1,1].add_patch(pyplot.Rectangle((-3,0),0.5, 1,facecolor='black',
                              clip_on=False,linewidth = 0))

#plt.tight_layout()
plt.show()

## Set up parameters and function for video export

In [ ]:
def testSubprocess():
    fontprops = fm.FontProperties(size=18)
    
    #set up the figure
    fig, axs = plt.subplots(2, 2, figsize=(18,16), gridspec_kw={'height_ratios': [3, 1]}, sharey="row")
    axs[0,0].axis('off')
    axs[0,1].axis('off')
    canvas_width, canvas_height = fig.canvas.get_width_height()

    
    scalebar1 = AnchoredSizeBar(axs[0,0].transData,
                           63.3, '100 µm', 'lower right', 
                           pad=0.1,
                           color='white',
                           frameon=False,
                           size_vertical=1,
                           fontproperties=fontprops)

    scalebar2 = AnchoredSizeBar(axs[0,1].transData,
                           63.3, '100 µm', 'lower right', 
                           pad=0.1,
                           color='white',
                           frameon=False,
                           size_vertical=1,
                           fontproperties=fontprops)
    
    axs[0,0].add_artist(scalebar1)
    axs[0,1].add_artist(scalebar2)
    
    # First frame
    fig_left = axs[0, 0].imshow(ndarr_left[0], ) # cmap="Greens_r"
    ts_left = axs[1,0].plot(tstamps_left_zeroed_s, time_series_left_zeroed)
    vline_left = axs[1,0].axvline(x=tstamps_left_zeroed_s[0], color="red")
    
    
    fig_right = axs[0, 1].imshow(ndarr_right[0], ) # cmap="Greens_r"
    ts_right = axs[1,1].plot(tstamps_right_zeroed_s, time_series_right_zeroed)
    vline_right = axs[1, 1].axvline(x=tstamps_right_zeroed_s[0], color="red")

    axs[0,1].set_title('AAV1.syn.Flex.GCaMP6s.WPRE.SV40\nAAV.CamKIIO.4.Cre SV40', y=1.0, pad=-42, color="white", fontsize=20)
    axs[0,0].set_title('AAV1.syn.GCaMP6s.WPRE.SV40', y=1.0, pad=-20, color="white", fontsize=20)

    axs[1,0].set_xlabel("Time (s)", fontsize=22)
    axs[1,0].tick_params(axis='x', labelsize=20)

    axs[1,1].set_xlabel("Time (s)", fontsize=22)
    axs[1,1].tick_params(axis='x', labelsize=20)

    axs[1,0].set_ylabel("100% df/f", fontsize=18)
    axs[1,1].set_ylabel("100% df/f", fontsize=18)
    #axs[1,0].tick_params(axis='y', labelsize=20)

    #axs[1,0].get_yaxis().set_visible(False)
    #axs[1,1].get_yaxis().set_visible(False)
    axs[1,0].get_yaxis().set_ticks([])
    axs[1,1].get_yaxis().set_ticks([])
    axs[1,0].spines['left'].set_visible(False)
    axs[1,0].spines['right'].set_visible(False)
    axs[1,0].spines['top'].set_visible(False)
    axs[1,1].spines['left'].set_visible(False)
    axs[1,1].spines['right'].set_visible(False)
    axs[1,1].spines['top'].set_visible(False)
    axs[1,0].add_patch(pyplot.Rectangle((-3,0),0.5, 1,facecolor='black',
                                  clip_on=False,linewidth = 0))
    axs[1,1].add_patch(pyplot.Rectangle((-3,0),0.5, 1,facecolor='black',
                                  clip_on=False,linewidth = 0))
    

    def update(frame):
        # your matplotlib code goes here
        fig_left.set_data(ndarr_left[frame])
        fig_right.set_data(ndarr_right[frame])
        vline_left.set_data([tstamps_left_zeroed_s[frame], tstamps_left_zeroed_s[frame]], [0, 1])
        vline_right.set_data([tstamps_right_zeroed_s[frame], tstamps_right_zeroed_s[frame]], [0, 1])

    # Open an ffmpeg process
    outf = "D:\\PhD\\Data\\testvid\\" + fh.get_filename_with_date("test_fast", ".mp4")
    print(outf)
    # lossless encoding:
    # https://stackoverflow.com/questions/37344997/how-to-get-a-lossless-encoding-with-ffmpeg-libx265
    cmdstring = ('ffmpeg', 
                 '-y', '-r', '15', # overwrite, 1fps
                 '-s', '%dx%d' % (canvas_width, canvas_height), # size of image string
                 '-pix_fmt', 'argb', # format
                 '-f', 'rawvideo',  '-i', '-', # tell ffmpeg to expect raw video from the pipe
                 #'-vcodec', 'mpeg4', outf) # use mpeg4 encoding
                 '-c:v', 'libx265',
                 '-x265-params', '"profile=monochrome12:crf=0:lossless=1:preset=veryslow:qp=0"',
                 outf)
    p = subprocess.Popen(cmdstring, stdin=subprocess.PIPE)

    # Draw frames and write to the pipe
    for frame in range(min(len(time_series_left), len(time_series_right))):
        print(frame)
        # draw the frame
        update(frame)
        fig.canvas.draw()

        # extract the image as an ARGB string
        string = fig.canvas.tostring_argb()
        
        # write to pipe
        p.stdin.write(string)

    # Finish up
    p.communicate()

In [ ]:
testSubprocess()

# Play Nikon recording, sliding bar of mean fluorescence, lfp, movement

In [ ]:
open_and_match = True  # if True, open nd2, labview etc. files and match them automatically. If False, open already matched h5 file. 

In [ ]:
FLUO_COLOR = "green"
LFP_COLOR = "black"
LOCO_COLOR = "grey"
BAR_COLOR = "red"
CMAP="viridis"

In [ ]:
if open_and_match:
    matlab_2p_folder = fh.open_dir("Open matlab-2p folder")

In [ ]:
if open_and_match:
    # TODO: switch to UUID
    nd2_fpath = fh.open_file("Open nd2 file")
    nd2_timestamps_fpath = fh.open_file("Open Nikon time stamp file")
    labview_fpath = fh.open_file("Open labview file")
    labview_timestamps_fpath = os.path.splitext(labview_fpath)[0] + "time.txt"
    lfp_fpath = fh.open_file("Open LFP file")
else:
    h5fpath = fh.open_file("Open h5 file containing TwoPhotonSession!") 

In [ ]:
if open_and_match:
    print(nd2_fpath)
    print(nd2_timestamps_fpath)
    print(labview_fpath)
    print(labview_timestamps_fpath)
    print(lfp_fpath)
else:
    print(h5fpath)

In [ ]:
if open_and_match:
    tps = TwoPhotonSession.init_and_process(nd2_fpath, nd2_timestamps_fpath, labview_fpath, labview_timestamps_fpath, lfp_fpath, matlab_2p_folder)

In [ ]:
if open_and_match:
    fluor = tps.return_nikon_mean()

In [ ]:
if open_and_match:
    t = tps.belt_dict["time"]
    t_fluor = tps.belt_dict["tsscn"]
    t_fluor_div = t_fluor/1000.
    loco = tps.belt_scn_dict["speed"] # tps.belt_dict["speed"]

In [ ]:
if open_and_match:
    t_lfp, y_lfp = tps.lfp_lfp()

In [ ]:
if open_and_match:
    lfp_y = tps.lfp_df_cut["y_lfp"]
    lfp_t = tps.lfp_df_cut["t_lfp_corrected"]
    loco_y = tps.lfp_df_cut["y_mov"]
    loco_t = tps.lfp_df_cut["t_mov_corrected"]

In [ ]:
if not open_and_match:
    with h5py.File(h5fpath, "r") as hf:
        lv_fpath = hf["basic"]["LABVIEW_PATH"][()].decode()
        lv_tstamps_fpath = hf["basic"]["LABVIEW_TIMESTAMPS_PATH"][()].decode()
        lfp_fpath = hf["basic"]["LFP_PATH"][()].decode()
        matlab_2p_folder = hf["basic"]["MATLAB_2P_FOLDER"][()].decode()
        nd2_fpath = hf["basic"]["ND2_PATH"][()].decode()
        nd2_tstamps_fpath = hf["basic"]["ND2_TIMESTAMPS_PATH"][()].decode()
        
        fluor = hf["mean_fluo_nucleus"][()]
        t_fluor = hf["inferred"]["belt_scn_dict"]["tsscn"][()]
        t_fluor_div = t_fluor/1000.
        loco = hf["inferred"]["belt_scn_dict"]["speed"][()]
        lfp_t = hf["inferred"]["lfp_df_cut"]["t_lfp_corrected"][()]
        lfp_y = hf["inferred"]["lfp_df_cut"]["y_lfp"][()]
        loco_y = hf["inferred"]["lfp_df_cut"]["y_mov"][()]
        loco_t = hf["inferred"]["lfp_df_cut"]["t_mov_corrected"][()]
        

In [ ]:
fig = plt.figure()
plt.plot(t_fluor/1000., loco)
plt.plot(loco_t, loco_y - 1.8)
plt.show()

In [ ]:
if open_and_match:
    nik = tps.get_nikon_data()
else:
    nik = np.array(pims_nd2.ND2_Reader(nd2_fpath)[:])

In [ ]:
plt.plot(fluor[4400:5500])
plt.ylim((20, 70))

In [ ]:
plt.plot(lfp_y)

In [ ]:
plt.plot(loco)

In [ ]:
# chr1: 
"""
ax_fluor.set_xlim([t_fluor_div[4450-5], t_fluor_div[5500+5]])
ax_lfp.set_xlim([t_fluor_div[4450-5], t_fluor_div[5500+1]])
ax_loco.set_xlim([t_fluor_div[4450-5], t_fluor_div[5500+1]])

ax_fluor.set_ylim([0, 150])
ax_lfp.set_ylim([-0.7, 0.25])
ax_loco.set_ylim([-0.12, 0.12])


saveVid(4450, 5500)
"""
# chr2:
"""
ax_fluor.set_xlim([t_fluor_div[4450-5], t_fluor_div[5500+5]])
ax_lfp.set_xlim([t_fluor_div[4450-5], t_fluor_div[5500+1]])
ax_loco.set_xlim([t_fluor_div[4450-5], t_fluor_div[5500+1]])


ax_fluor.set_ylim([0, 100])
ax_lfp.set_ylim([-0.75, 0.25])
ax_loco.set_ylim([-0.08, 0.6])  # used 0.4 sometimes, leads to clipping during stim


saveVid(4450, 5500)
"""

In [ ]:
# Layout:
# left top figure: average luminosity
# Left middle: LFP
# left bottom: locomotion 
# Right: nd2 video playing

i_frame = 10
fontprops = fm.FontProperties(size=18)
#set up the figure

# height ratios:
# https://stackoverflow.com/questions/10388462/matplotlib-different-size-subplots
# update matplotlib, then height_ratios can be directly passed to subplots()!
#fig, axs = plt.subplots(3, 2, figsize=(18,10), gridspec_kw={'height_ratios': [3, 1]}, sharey="row")
#fig, axs = plt.subplots(3, 2, figsize=(18,10), sharey="row")
#axs[0,0].axis('off')
#axs[0,1].axis('off')
#canvas_width, canvas_height = fig.canvas.get_width_height()

# 1.579 µm / pixel -> 100 µm = 63.33 pixels
"""
scalebar1 = AnchoredSizeBar(axs[0,0].transData,
                           63.3, '100 µm', 'lower right', 
                           pad=0.1,
                           color='white',
                           frameon=False,
                           size_vertical=1,
                           fontproperties=fontprops)

scalebar2 = AnchoredSizeBar(axs[0,1].transData,
                           63.3, '100 µm', 'lower right', 
                           pad=0.1,
                           color='white',
                           frameon=False,
                           size_vertical=1,
                           fontproperties=fontprops)


axs[0,0].add_artist(scalebar1)
axs[0,1].add_artist(scalebar2)
"""
# Fill first frame
#fig_right = axs[0, 1].imshow(ndarr_right[i_frame], )  #cmap="Greens_r"
#ts_right = axs[1,1].plot(tstamps_right_zeroed_s, time_series_right_zeroed)
#vline_right = axs[1, 1].axvline(x=tstamps_right_zeroed_s[i_frame], color="red")

#fluor_plot = axs[0,0].plot(t_fluor, fluor)
#loco_plot = axs[1,0].plot(t, loco)
#lfp_plot = axs[2, 0].plot(t_lfp, y_lfp)

t_vertline = t_fluor_div[4800]

fig = plt.figure(figsize=(18,10))
canvas_width, canvas_height = fig.canvas.get_width_height()
gridspec = fig.add_gridspec(3, 2, height_ratios = [3, 3, 1], width_ratios=[1,1])
ax_loco = fig.add_subplot(gridspec[2,0])
ax_fluor = fig.add_subplot(gridspec[0,0], sharex=ax_loco)
ax_lfp =  fig.add_subplot(gridspec[1,0], sharex=ax_loco)

ax_nikon = fig.add_subplot(gridspec[:, 1])

ax_fluor.plot(t_fluor_div, fluor, color=FLUO_COLOR, linewidth=3)
ax_lfp.plot(lfp_t, lfp_y, color=LFP_COLOR, linewidth=0.1)
ax_loco.plot(t_fluor_div, loco, color=LOCO_COLOR, linewidth=3)


fig_nikon = ax_nikon.imshow(nik[0], cmap=CMAP)

#vline_fluor = ax_fluor.axvline(x=t_vertline, ymin=-1.2, ymax=1, zorder=0,clip_on=False, color="black")
#vline_lfp = ax_lfp.axvline(x=t_vertline, ymin=-1.2, ymax=1, zorder=0,clip_on=False, color="black")
vline_loco = ax_loco.axvline(x=t_vertline, ymin=0, ymax=8, zorder=0,clip_on=False, color=BAR_COLOR, linewidth=3)

fig_nikon.set_data(nik[100])

ax_nikon.axis('off')
ax_fluor.axis('off')
ax_lfp.axis('off')
ax_loco.spines['left'].set_visible(False)
ax_loco.spines['right'].set_visible(False)
ax_loco.spines['top'].set_visible(False)
ax_loco.get_yaxis().set_ticks([])


#ax_fluor.set_xlim([t_fluor_div[4450-5], t_fluor_div[5500+5]])
#ax_lfp.set_xlim([t_fluor_div[4450-5], t_fluor_div[5500+1]])
ax_loco.set_xlim([t_fluor_div[4450-5], t_fluor_div[5500+1]])
#ax_loco.set_xlim([300, 310])


ax_fluor.set_ylim([0, 210]) 
ax_lfp.set_ylim([0.1, 0.25]) 
ax_loco.set_ylim([-0.3, 1.2]) 

ax_loco.tick_params(axis='x', labelsize=18)
ax_loco.set_xlabel("Time (s)", fontsize=20)

#ax_fluor.get_yaxis().set_visible(False)
#ax_lfp.get_yaxis().set_visible(False)
#ax_loco.get_yaxis().set_visible(False)

#ax_fluor.get_xaxis().set_visible(False)
#ax_lfp.get_xaxis().set_visible(False)

plt.show()

In [ ]:
def saveVid(i_frame_begin, i_frame_end):  # in 1-indexing
    t_vertline = t_fluor[i_frame_begin - 1]
    VERTBAR_HEIGHT = 8.  # adjust this based on gridspec height_ratios and other stuff... 
                         # For equal-sized three plots, 3.5 used to be enough. For 3:3:1, 8. was needed
    
    
    fig = plt.figure(figsize=(18,10))
    canvas_width, canvas_height = fig.canvas.get_width_height()
    gridspec = fig.add_gridspec(3, 2, height_ratios = [3, 3, 1])
    ax_fluor = fig.add_subplot(gridspec[0,0])
    ax_lfp =  fig.add_subplot(gridspec[1,0])
    ax_loco = fig.add_subplot(gridspec[2,0])
    ax_nikon = fig.add_subplot(gridspec[:, 1])

    ax_fluor.plot(t_fluor/1000., fluor, color=FLUO_COLOR, linewidth=3)
    ax_lfp.plot(lfp_t, lfp_y, color=LFP_COLOR, linewidth=3)
    #ax_loco.plot(loco_t, loco_y, color="red")
    ax_loco.plot(t_fluor_div, loco, color=LOCO_COLOR, linewidth=3)
    
    fig_nikon = ax_nikon.imshow(nik[0], cmap=CMAP)

    #vline_fluor = ax_fluor.axvline(x=t_vertline, ymin=-1.2, ymax=1, zorder=0,clip_on=False, color="black")
    #vline_lfp = ax_lfp.axvline(x=t_vertline, ymin=-1.2, ymax=1, zorder=0,clip_on=False, color="black")
    vline_loco = ax_loco.axvline(x=t_vertline, ymin=0, ymax=VERTBAR_HEIGHT, zorder=0,clip_on=False, color=BAR_COLOR, linewidth=3)
    
    ax_fluor.set_xlim([t_fluor_div[i_frame_begin-5], t_fluor_div[i_frame_end+5]])
    ax_lfp.set_xlim([t_fluor_div[i_frame_begin-5], t_fluor_div[i_frame_end+1]])
    ax_loco.set_xlim([t_fluor_div[i_frame_begin-5], t_fluor_div[i_frame_end+1]])
    
    ax_fluor.set_ylim([0, 170])
    ax_lfp.set_ylim([-0.75, 1.])
    ax_loco.set_ylim([-0.5, 1.5])
    
    ax_nikon.axis('off')
    ax_fluor.axis('off')
    ax_lfp.axis('off')
    ax_loco.spines['left'].set_visible(False)
    ax_loco.spines['right'].set_visible(False)
    ax_loco.spines['top'].set_visible(False)
    ax_loco.get_yaxis().set_ticks([])
    
    ax_loco.tick_params(axis='x', labelsize=18)
    ax_loco.set_xlabel("Time (s)", fontsize=20)
    
    
    def update(frame):
        # your matplotlib code goes here
        fig_nikon.set_data(nik[frame])
        
        vline_loco.set_data([t_fluor_div[frame - 1], t_fluor_div[frame - 1]], [0, VERTBAR_HEIGHT])

    # Open an ffmpeg process
    outf = os.path.join(fh.open_dir("Choose output folder"), fh.get_filename_with_date("test_video", ".mp4"))
    print(outf)
    # lossless encoding:
    # https://stackoverflow.com/questions/37344997/how-to-get-a-lossless-encoding-with-ffmpeg-libx265
    cmdstring = ('ffmpeg', 
                 '-y', '-r', '15', # overwrite, 1fps
                 '-s', '%dx%d' % (canvas_width, canvas_height), # size of image string
                 '-pix_fmt', 'argb', # format
                 '-f', 'rawvideo',  '-i', '-', # tell ffmpeg to expect raw video from the pipe
                 #'-vcodec', 'mpeg4', outf) # use mpeg4 encoding
                 '-c:v', 'libx265',
                 '-x265-params', '"profile=monochrome12:crf=0:lossless=1:preset=veryslow:qp=0"',
                 outf)
    p = subprocess.Popen(cmdstring, stdin=subprocess.PIPE, shell=True)

    # Draw frames and write to the pipe
    for frame in range(i_frame_begin, i_frame_end+1):
        print(frame)
        # draw the frame
        update(frame)
        fig.canvas.draw()

        # extract the image as an ARGB string
        string = fig.canvas.tostring_argb()
        # write to pipe
        p.stdin.write(string)

    # Finish up
    p.communicate()

In [ ]:
plt.imshow(nik[0], cmap="gray")

In [ ]:
def saveVidWindow(i_frame_begin, i_frame_end, window_size, vid_fps=15, step_size=1, cmap="gray"):  
    VERTBAR_HEIGHT = 8.  # adjust this based on gridspec height_ratios and other stuff... 
                         # For equal-sized three plots, 3.5 used to be enough. For 3:3:1, 8. was needed
    #window_size denotes the size of the individual segments to show in the video for the sliding bar!
    #step_size needs to be manually adjusted along with vid_fps, to get the desired video. step_size denotes the 
    #jump between two frames (a default of 1 means every frame within the range is recorded as a frame). To double the speed,
    # set step_size to 2 etc.
    # i_frame_begin and i_frame_end in 1-indexing.i.e. write 1 if from first frame.
    # window_size is in frames! For 15 Hz, 10 s should be window_size=150. 
    # Need to adjust window_size and vid_fps by hand to get desired results!
    t_vertline = t_fluor_div[i_frame_begin - 1]
    # create tuples of begin and end frames. All should have same size to preserve x-axis width.
    intervals = [(i_frame-1, i_frame-1 + window_size) for i_frame in range(i_frame_begin, i_frame_end, window_size)]
    intervals_s = [(t_fluor_div[i[0]], t_fluor_div[i[1]]) for i in intervals]
    
    fig = plt.figure(figsize=(18,10))
    canvas_width, canvas_height = fig.canvas.get_width_height()
    gridspec = fig.add_gridspec(3, 2, height_ratios = [3, 3, 1])
    ax_loco = fig.add_subplot(gridspec[2,0])
    ax_fluor = fig.add_subplot(gridspec[0,0], sharex=ax_loco)
    ax_lfp =  fig.add_subplot(gridspec[1,0], sharex=ax_loco)
    ax_nikon = fig.add_subplot(gridspec[:, 1])

    ax_fluor.plot(t_fluor_div, fluor, color=FLUO_COLOR, linewidth=3)
    ax_lfp.plot(lfp_t, lfp_y, color=LFP_COLOR, linewidth=0.3)
    #ax_loco.plot(loco_t, loco_y, color="red")
    ax_loco.plot(t_fluor_div, loco, color=LOCO_COLOR, linewidth=3)
    
    fig_nikon = ax_nikon.imshow(nik[0], cmap=cmap)

    #vline_fluor = ax_fluor.axvline(x=t_vertline, ymin=-1.2, ymax=1, zorder=0,clip_on=False, color="black")
    #vline_lfp = ax_lfp.axvline(x=t_vertline, ymin=-1.2, ymax=1, zorder=0,clip_on=False, color="black")
    vline_loco = ax_loco.axvline(x=t_vertline, ymin=0, ymax=VERTBAR_HEIGHT, zorder=0,clip_on=False, color=BAR_COLOR, linewidth=3)
    
    ax_loco.set_xlim(intervals_s[0])
    
    # edit these values to adjust for each individual recording

    ax_fluor.set_ylim([20, 70]) 
    ax_lfp.set_ylim([-0.4, 0.5])
    ax_loco.set_ylim([-0.2, 0.8]) 

    ax_nikon.axis('off')
    ax_fluor.axis('off')
    ax_lfp.axis('off')
    ax_loco.spines['left'].set_visible(False)
    ax_loco.spines['right'].set_visible(False)
    ax_loco.spines['top'].set_visible(False)
    ax_loco.get_yaxis().set_ticks([])
    
    ax_loco.tick_params(axis='x', labelsize=18)
    ax_loco.set_xlabel("Time (s)", fontsize=20)
    
    
    def update(frame):
        # your matplotlib code goes here
        fig_nikon.set_data(nik[frame])
        for i_interv, interv in enumerate(intervals):
            if frame in range(*interv):
                ax_loco.set_xlim([intervals_s[i_interv][0], intervals_s[i_interv][1]])
                break
        
        vline_loco.set_data([t_fluor_div[frame - 1], t_fluor_div[frame - 1]], [0, VERTBAR_HEIGHT])

    # Open an ffmpeg process
    outf = os.path.join(fh.open_dir("Choose output folder"), fh.get_filename_with_date("test_video", ".mp4"))
    print(outf)
    # lossless encoding:
    # https://stackoverflow.com/questions/37344997/how-to-get-a-lossless-encoding-with-ffmpeg-libx265
    # Original, lossless version. Big size, good quality?
    cmdstring = ('ffmpeg', 
                 '-y', '-r', str(vid_fps), # overwrite, 15fps
                 '-s', '%dx%d' % (canvas_width, canvas_height), # size of image string
                 '-pix_fmt', 'argb', # format
                 '-f', 'rawvideo',  '-i', '-', # tell ffmpeg to expect raw video from the pipe
                 #'-vcodec', 'mpeg4', outf) # use mpeg4 encoding
                 '-c:v', 'libx265',
                 '-x265-params', '"profile=monochrome12:crf=0:lossless=1:preset=veryslow:qp=0"',
                 outf)
    """
    # A failed version of x264 lossless.
    cmdstring = ('ffmpeg', 
             '-y', '-r', str(vid_fps), # overwrite, 15fps
             '-s', '%dx%d' % (canvas_width, canvas_height), # size of image string
             '-pix_fmt', 'argb', # format
             '-f', 'rawvideo',  '-i', '-', # tell ffmpeg to expect raw video from the pipe
             '-vcodec', 'libx264', outf) # use mpeg4 encoding
             #'-c:v', 'libx264',
             #'-x264-params', '"profile=monochrome12:crf=0:lossless=1:preset=veryslow:qp=0"',
             #outf)
    """
    """
    # This is yet another failed version, compressed
    cmdstring = ('ffmpeg', 
                 '-y', '-r', str(vid_fps), # overwrite, 15fps
                 '-s', '%dx%d' % (canvas_width, canvas_height), # size of image string
                 '-f', 'rawvideo',  '-i', '-', # tell ffmpeg to expect raw video from the pipe
                 '-c:v', 'libx264',
                 '-crf', '23',
                 #'profile:v', 'baseline',
                 #'-level', '3.0',
                 '-pix_fmt', 'yuv420p', # format
                 '-c:a', 'aac',
                 '-ac', '2', 
                 '-b:a', '128k',
                 '-movflags', 'faststart',
                 outf)
    """
    print(cmdstring)
    p = subprocess.Popen(cmdstring, stdin=subprocess.PIPE, shell=True)  
    # TODO: shell=True is sometimes not advised to do... Might want to try without

    # Draw frames and write to the pipe
    for frame in range(i_frame_begin, i_frame_end+1, step_size):
        print(frame)
        # draw the frame
        update(frame)
        fig.canvas.draw()

        # extract the image as an ARGB string
        string = fig.canvas.tostring_argb()
        # write to pipe
        p.stdin.write(string) 

    # Finish up
    p.communicate()

In [ ]:
#saveVid(10210, 12000)

# 120 Hz recording. To get 30 Hz video, need to take every 4th frame.For 6s windows, need a window size of 120*6=720
saveVidWindow(4400, 5500, 720, 15, 1, "viridis")   # for jedi g184 050723 1505
# 8000 - 23000
# 420 Hz, to get 30Hz video, need to take every 14th frame. Choose 7s windows, 2940 frames per window. 
#saveVidWindow(8000, 23000, 2940, 30, 14)  # for jedi g184 050723 1440
# saveVidWindow(4450, 4600, 50)  # for debugging; small videos

In [ ]:
# Layout:
# left top figure: average luminosity
# Left middle: LFP
# left bottom: locomotion 
# Right: nd2 video playing

i_frame = 10
fontprops = fm.FontProperties(size=18)
#set up the figure

# height ratios:
# https://stackoverflow.com/questions/10388462/matplotlib-different-size-subplots
# update matplotlib, then height_ratios can be directly passed to subplots()!
#fig, axs = plt.subplots(3, 2, figsize=(18,10), gridspec_kw={'height_ratios': [3, 1]}, sharey="row")
#fig, axs = plt.subplots(3, 2, figsize=(18,10), sharey="row")
#axs[0,0].axis('off')
#axs[0,1].axis('off')
#canvas_width, canvas_height = fig.canvas.get_width_height()

# 1.579 µm / pixel -> 100 µm = 63.33 pixels
"""
scalebar1 = AnchoredSizeBar(axs[0,0].transData,
                           63.3, '100 µm', 'lower right', 
                           pad=0.1,
                           color='white',
                           frameon=False,
                           size_vertical=1,
                           fontproperties=fontprops)

scalebar2 = AnchoredSizeBar(axs[0,1].transData,
                           63.3, '100 µm', 'lower right', 
                           pad=0.1,
                           color='white',
                           frameon=False,
                           size_vertical=1,
                           fontproperties=fontprops)


axs[0,0].add_artist(scalebar1)
axs[0,1].add_artist(scalebar2)
"""
# Fill first frame
#fig_right = axs[0, 1].imshow(ndarr_right[i_frame], )  #cmap="Greens_r"
#ts_right = axs[1,1].plot(tstamps_right_zeroed_s, time_series_right_zeroed)
#vline_right = axs[1, 1].axvline(x=tstamps_right_zeroed_s[i_frame], color="red")

#fluor_plot = axs[0,0].plot(t_fluor, fluor)
#loco_plot = axs[1,0].plot(t, loco)
#lfp_plot = axs[2, 0].plot(t_lfp, y_lfp)

t_vertline = 200.0

fig = plt.figure(figsize=(18,10))
canvas_width, canvas_height = fig.canvas.get_width_height()
gridspec = fig.add_gridspec(3, 2)
ax_fluor = fig.add_subplot(gridspec[0,0])
ax_lfp =  fig.add_subplot(gridspec[1,0])
ax_loco = fig.add_subplot(gridspec[2,0])
ax_nikon = fig.add_subplot(gridspec[:, 1])

ax_fluor.plot(t_fluor/1000., fluor, color="green")
ax_lfp.plot(t_lfp, y_lfp, color="blue")
ax_loco.plot(t/1000., loco, color="red")

fig_nikon = ax_nikon.imshow(nik[0])

vline_fluor = ax_fluor.axvline(x=t_vertline, color="black")
vline_lfp = ax_lfp.axvline(x=t_vertline, color="black")
vline_loco = ax_loco.axvline(x=t_vertline, color="black")

ax_nikon.axis('off')
ax_fluor.axis('off')
ax_lfp.axis('off')
ax_loco.spines['left'].set_visible(False)
ax_loco.spines['right'].set_visible(False)
ax_loco.spines['top'].set_visible(False)
ax_loco.get_yaxis().set_ticks([])
#ax_fluor.get_yaxis().set_visible(False)
#ax_lfp.get_yaxis().set_visible(False)
#ax_loco.get_yaxis().set_visible(False)

#ax_fluor.get_xaxis().set_visible(False)
#ax_lfp.get_xaxis().set_visible(False)

plt.show()

In [ ]:
lfp_mov_t, lfp_mov_y = tps.lfp_movement()

In [ ]:
plt.plot(lfp_mov_t, lfp_mov_y)

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(18, 18))

ax1.plot(tps.lfp_df_cut.t_mov_corrected, tps.lfp_df_cut.y_mov_normalized)
ax2.plot(tps.belt_df["time_s"], tps.belt_df["speed"])
#ax2.set_xlim([1112, 1115])
ax2.set_xlim([325, 335])
plt.show()

# Create vector graphics from aligned traces
### Specifically for two recordings here. Very early stage code

In [ ]:
FLUO_COLOR = "green"
LFP_COLOR = "black"
LOCO_COLOR = "grey"
BAR_COLOR = "red"

In [ ]:
matlab_2p_folder = fh.open_dir("Open matlab-2p folder")

In [ ]:
# TODO: switch to UUID
nd2_fpath = fh.open_file("Open nd2 file")
nd2_timestamps_fpath = fh.open_file("Open Nikon time stamp file")
labview_fpath = fh.open_file("Open labview file")
labview_timestamps_fpath = os.path.splitext(labview_fpath)[0] + "time.txt"
lfp_fpath = fh.open_file("Open LFP file")

In [ ]:
tps = TwoPhotonSession.init_and_process(nd2_fpath, nd2_timestamps_fpath, labview_fpath, labview_timestamps_fpath, lfp_fpath, matlab_2p_folder)

In [ ]:
fluor = tps.return_nikon_mean()

In [ ]:
t = tps.belt_dict["time"]
t_fluor = tps.belt_dict["tsscn"]
t_fluor_div = t_fluor/1000.
loco = tps.belt_scn_dict["speed"] # tps.belt_dict["speed"]

In [ ]:
nik = tps.get_nikon_data()

In [ ]:
lfp_y = tps.lfp_df_cut["y_lfp"]
lfp_t = tps.lfp_df_cut["t_lfp_corrected"]
loco_y = tps.lfp_df_cut["y_mov"]
loco_t = tps.lfp_df_cut["t_mov_corrected"]

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(18,10), gridspec_kw={'height_ratios': [3,3,1]}, sharex="col")
axs[0].plot(t_fluor_div, fluor, color=FLUO_COLOR)
axs[1].plot(lfp_t, lfp_y, color=LFP_COLOR, linewidth=0.3)
axs[2].plot(t_fluor_div, loco, color=LOCO_COLOR)

axs[0].axis("off")
axs[1].axis("off")

axs[2].spines['left'].set_visible(False)
axs[2].spines['right'].set_visible(False)
axs[2].spines['top'].set_visible(False)
axs[2].get_yaxis().set_ticks([])

axs[2].tick_params(axis='x', labelsize=18)
axs[2].set_xlabel("Time (s)", fontsize=20)

t_begin = 290
t_end = 370
#axs[0].set_xlim(t_begin, t_end)  # x axes are shared in col
#axs[1].set_xlim(t_begin, t_end)  # x axes are shared in col
axs[2].set_xlim(t_begin, t_end)

axs[0].set_ylim(20, 127)
axs[1].set_ylim(-0.7, 0.32)
axs[2].set_ylim(-0.15, 0.1)


plt.show()
#plt.savefig('D:\\PhD\\Traces\\T386_d6_1201.eps', format='eps')

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(18,10), gridspec_kw={'height_ratios': [3,3,1]})  # sharex="col" makes shifting easy,
# but makes hiding x ticks on upper plots difficult while keeping bottom ticks.

axs[0].plot(t_fluor_div, fluor, color=FLUO_COLOR)
axs[1].plot(lfp_t, lfp_y, color=LFP_COLOR, linewidth=0.3)
axs[2].plot(t_fluor_div, loco, color=LOCO_COLOR)


t_begin = 290
t_end = 370
axs[0].set_xlim(t_begin, t_end)
axs[1].set_xlim(t_begin, t_end)
axs[2].set_xlim(t_begin, t_end)

axs[0].set_ylim(20, 127)
axs[1].set_ylim(-0.7, 0.32)
axs[2].set_ylim(-0.15, 0.1)

# Fluorescence plot
axs[0].set_ylabel("mean fluorescense (a.u.)", fontsize=15, labelpad=20)
axs[0].spines['left'].set_visible(False)
axs[0].spines['right'].set_visible(False)
axs[0].spines['top'].set_visible(False)
axs[0].spines['bottom'].set_visible(False)
axs[0].get_yaxis().set_ticks([])
axs[0].get_xaxis().set_ticks([])

# LFP plot
axs[1].set_ylabel("LFP (a.u.)", fontsize=15, labelpad=20)
axs[1].spines['left'].set_visible(False)
axs[1].spines['right'].set_visible(False)
axs[1].spines['top'].set_visible(False)
axs[1].spines['bottom'].set_visible(False)
axs[1].get_yaxis().set_ticks([])
axs[1].get_xaxis().set_ticks([])


# Loco plot
axs[2].set_ylabel("locomotion velocity (a.u.)", fontsize=15, labelpad=20)
axs[2].spines['left'].set_visible(False)
axs[2].spines['right'].set_visible(False)
axs[2].spines['top'].set_visible(False)
axs[2].get_yaxis().set_ticks([])
axs[2].tick_params(axis='x', labelsize=18)
axs[2].set_xlabel("Time (s)", fontsize=20)

plt.show()   
#plt.savefig('D:\\PhD\\Traces\\T386_d6_1201_labeled.eps', format='eps')

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(18,10), gridspec_kw={'height_ratios': [3,3,1]})  # sharex="col" makes shifting easy,
# but makes hiding x ticks on upper plots difficult while keeping bottom ticks.

axs[0].plot(t_fluor_div, fluor, color=FLUO_COLOR)
axs[1].plot(lfp_t, lfp_y, color=LFP_COLOR, linewidth=0.3)
axs[2].plot(t_fluor_div, loco, color=LOCO_COLOR)


t_begin = 290
t_end = 370
axs[0].set_xlim(t_begin, t_end)
axs[1].set_xlim(t_begin, t_end)
axs[2].set_xlim(t_begin, t_end)

axs[0].set_ylim(20, 127)
axs[1].set_ylim(-0.7, 0.32)
axs[2].set_ylim(-0.15, 0.5)

# Fluorescence plot
axs[0].set_ylabel("mean fluorescense (a.u.)", fontsize=15, labelpad=20)
axs[0].spines['left'].set_visible(False)
axs[0].spines['right'].set_visible(False)
axs[0].spines['top'].set_visible(False)
axs[0].spines['bottom'].set_visible(False)
axs[0].get_yaxis().set_ticks([])
axs[0].get_xaxis().set_ticks([])

# LFP plot
axs[1].set_ylabel("LFP (a.u.)", fontsize=15, labelpad=20)
axs[1].spines['left'].set_visible(False)
axs[1].spines['right'].set_visible(False)
axs[1].spines['top'].set_visible(False)
axs[1].spines['bottom'].set_visible(False)
axs[1].get_yaxis().set_ticks([])
axs[1].get_xaxis().set_ticks([])


# Loco plot
axs[2].set_ylabel("locomotion velocity (a.u.)", fontsize=15, labelpad=20)
axs[2].spines['left'].set_visible(False)
axs[2].spines['right'].set_visible(False)
axs[2].spines['top'].set_visible(False)
axs[2].get_yaxis().set_ticks([])
axs[2].tick_params(axis='x', labelsize=18)
axs[2].set_xlabel("Time (s)", fontsize=20)

#plt.show()   
plt.savefig('D:\\PhD\\Traces\\T386_d6_1201_labeled_common-y.eps', format='eps')

## Open other recording, assign same variables (just below plotting cells above). The difference is x limits and y limits.

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(18,10), gridspec_kw={'height_ratios': [3,3,1]}, sharex="col")
axs[0].plot(t_fluor_div, fluor, color=FLUO_COLOR)
axs[1].plot(lfp_t, lfp_y, color=LFP_COLOR, linewidth=0.3)
axs[2].plot(t_fluor_div, loco, color=LOCO_COLOR)

axs[0].axis("off")
axs[1].axis("off")

axs[2].spines['left'].set_visible(False)
axs[2].spines['right'].set_visible(False)
axs[2].spines['top'].set_visible(False)
axs[2].get_yaxis().set_ticks([])

axs[2].tick_params(axis='x', labelsize=18)
axs[2].set_xlabel("Time (s)", fontsize=20)

t_begin = 290
t_end = 370
#axs[0].set_xlim(t_begin, t_end)  # x axes are shared in col
#axs[1].set_xlim(t_begin, t_end)  # x axes are shared in col
axs[2].set_xlim(t_begin, t_end)

axs[0].set_ylim(25, 95)
axs[1].set_ylim(-0.7, 0.3)
axs[2].set_ylim(-0.08, 0.5)


#plt.show()
plt.savefig('D:\\PhD\\Traces\\T386_d5_1216.eps', format='eps')

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(18,10), gridspec_kw={'height_ratios': [3,3,1]})  # sharex="col" makes shifting easy,
# but makes hiding x ticks on upper plots difficult while keeping bottom ticks.

axs[0].plot(t_fluor_div, fluor, color=FLUO_COLOR)
axs[1].plot(lfp_t, lfp_y, color=LFP_COLOR, linewidth=0.3)
axs[2].plot(t_fluor_div, loco, color=LOCO_COLOR)


t_begin = 290
t_end = 370
axs[0].set_xlim(t_begin, t_end)
axs[1].set_xlim(t_begin, t_end)
axs[2].set_xlim(t_begin, t_end)

axs[0].set_ylim(25, 95)
axs[1].set_ylim(-0.7, 0.3)
axs[2].set_ylim(-0.08, 0.5)

# Fluorescence plot
axs[0].set_ylabel("mean fluorescense (a.u.)", fontsize=15, labelpad=20)
axs[0].spines['left'].set_visible(False)
axs[0].spines['right'].set_visible(False)
axs[0].spines['top'].set_visible(False)
axs[0].spines['bottom'].set_visible(False)
axs[0].get_yaxis().set_ticks([])
axs[0].get_xaxis().set_ticks([])

# LFP plot
axs[1].set_ylabel("LFP (a.u.)", fontsize=15, labelpad=20)
axs[1].spines['left'].set_visible(False)
axs[1].spines['right'].set_visible(False)
axs[1].spines['top'].set_visible(False)
axs[1].spines['bottom'].set_visible(False)
axs[1].get_yaxis().set_ticks([])
axs[1].get_xaxis().set_ticks([])


# Loco plot
axs[2].set_ylabel("locomotion velocity (a.u.)", fontsize=15, labelpad=20)
axs[2].spines['left'].set_visible(False)
axs[2].spines['right'].set_visible(False)
axs[2].spines['top'].set_visible(False)
axs[2].get_yaxis().set_ticks([])
axs[2].tick_params(axis='x', labelsize=18)
axs[2].set_xlabel("Time (s)", fontsize=20)

#plt.show()   
plt.savefig('D:\\PhD\\Traces\\T386_d5_1216_labeled.eps', format='eps')

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(18,10), gridspec_kw={'height_ratios': [3,3,1]})  # sharex="col" makes shifting easy,
# but makes hiding x ticks on upper plots difficult while keeping bottom ticks.

axs[0].plot(t_fluor_div, fluor, color=FLUO_COLOR)
axs[1].plot(lfp_t, lfp_y, color=LFP_COLOR, linewidth=0.3)
axs[2].plot(t_fluor_div, loco, color=LOCO_COLOR)


t_begin = 290
t_end = 370
axs[0].set_xlim(t_begin, t_end)
axs[1].set_xlim(t_begin, t_end)
axs[2].set_xlim(t_begin, t_end)

axs[0].set_ylim(20, 127)
axs[1].set_ylim(-0.7, 0.32)
axs[2].set_ylim(-0.15, 0.5)


# Fluorescence plot
axs[0].set_ylabel("mean fluorescense (a.u.)", fontsize=15, labelpad=20)
axs[0].spines['left'].set_visible(False)
axs[0].spines['right'].set_visible(False)
axs[0].spines['top'].set_visible(False)
axs[0].spines['bottom'].set_visible(False)
#axs[0].get_yaxis().set_ticks([])
axs[0].get_xaxis().set_ticks([])

# LFP plot
axs[1].set_ylabel("LFP (a.u.)", fontsize=15, labelpad=20)
axs[1].spines['left'].set_visible(False)
axs[1].spines['right'].set_visible(False)
axs[1].spines['top'].set_visible(False)
axs[1].spines['bottom'].set_visible(False)
#axs[1].get_yaxis().set_ticks([])
axs[1].get_xaxis().set_ticks([])


# Loco plot
axs[2].set_ylabel("locomotion velocity (a.u.)", fontsize=15, labelpad=20)
axs[2].spines['left'].set_visible(False)
axs[2].spines['right'].set_visible(False)
axs[2].spines['top'].set_visible(False)
#axs[2].get_yaxis().set_ticks([])
axs[2].tick_params(axis='x', labelsize=18)
axs[2].set_xlabel("Time (s)", fontsize=20)

plt.show()   
#plt.savefig('D:\\PhD\\Traces\\T386_d5_1216_labeled_common-y.eps', format='eps')